In [1]:
from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})

import csv
import torch
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import math
import pandas as pd

from Tarjet import *
from Phisicsparams import *
from utils import *
from MTMM import *
from tqdm import tqdm

from multiresglonet import GLOnet
from material_database import MatDatabase


In [22]:
from typing import TypeVarTuple


params = Params()
params.thickness_sup = 0.1
params.N_layers = 20

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params.materials = ['Al2O3','TiO2', 'SiO2']
params.user_define = True
if params.user_define:
  params.n_min = 1.09
  params.n_max = 2.6
  params.M_discretion_n = 100
  params.M_materials = params.M_discretion_n
  params.n_database = torch.tensor(np.array([np.linspace(params.n_min,params.n_max,params.M_discretion_n)]))
else:
  pass # definirlo en otro lado
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

params.alpha_sup =  15
params.numIter = 200
params.sigma = 0.035
params.batch_size = 100
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params.net = 'Res'
params.res_layers = 16                                                                             # Cantidad de bloques Residuales del bloque ResNet
params.res_dim = 256                                                                               # Cantidad de neuronas en la capa de entrada al bloque ResNet
params.noise_dim = 26                                                                              # Dimension de la Capa de entrada
params.lr = 0.05                                                                                   # Tasa de aprendizaje del optimizador Adam (learning rate)
params.beta1 = 0.9                                                                                 # Coeficiente de decaimiento para el momento del primer orden del optimizador Adam
params.beta2 = 0.99                                                                                # Coeficiente de decaimiento para el momento del segundo orden del optimizador Adam
params.weight_decay = 0.001                                                                        # Termino de decaimiento del peso para regularizar los pesos del generador durante la optimizacion
params.step_size = 40000                                                                           # Numero de epicas despues de las cuales se reduce la tasa de aprendizaje
params.gamma = 0.5                                                                                 # El factor de reduccion para la tasa de aprendizaje. Despues de cada step_size epocas, la tasa de aprendizaje se multiplica por gamma


In [23]:
params.condiciones = 2
physicsparams = PhysicsParams(params.condiciones, user_define=True)

n_interna = 1.2

physicsparams.n_bot_1 = n_interna
physicsparams.n_top_1 = 1
physicsparams.k_1 = 370
physicsparams.k_values.append(physicsparams.k_1)
physicsparams.theta_1 = 0
physicsparams.lambda_min_1 = 380
physicsparams.lambda_max_1 = 750
physicsparams.pol_1 = "TE"

physicsparams.n_bot_2 = 1
physicsparams.n_top_2 = n_interna
physicsparams.k_2 = 190
physicsparams.k_values.append(physicsparams.k_2)
physicsparams.theta_2 = 45
physicsparams.lambda_min_2 = 380
physicsparams.lambda_max_2 = 570
physicsparams.pol_2 = "TE"

physicsparams.generate_physics_params()
# physicsparams.view_attributes()

tarjet = Tarjet(params.condiciones, physicsparams.k_values, params.user_define)
tarjet.configure_targets()



In [24]:
if not params.user_define:
    params.matdatabase =  MatDatabase(params.materials)
    params.n_database = params.matdatabase.interp_wv(2 * math.pi/physicsparams.ks, params.materials, True)
    params.M_materials =  params.n_database.size(0)

In [7]:
from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})

import csv
import torch
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import math
import pandas as pd

from Tarjet import *
from Phisicsparams import *
from utils import *
from MTMM import *
from tqdm import tqdm

from multiresglonet import GLOnet
from material_database import MatDatabase

params = Params()
params.thickness_sup = 0.3
params.N_layers = 7

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params.materials = ['Al2O3','TiO2', 'SiO2']
params.user_define = True
if params.user_define:
  params.n_min = 1.09
  params.n_max = 2.6
  params.M_discretion_n = 100
  params.M_materials = params.M_discretion_n
  params.n_database = torch.tensor(np.array([np.linspace(params.n_min,params.n_max,params.M_discretion_n)]))
else:
  pass # definirlo en otro lado
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

params.alpha_sup =  15
params.numIter = 200
params.sigma = 0.035
params.batch_size = 150
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params.net = 'Res'
params.res_layers = 16                                                                             # Cantidad de bloques Residuales del bloque ResNet
params.res_dim = 256                                                                               # Cantidad de neuronas en la capa de entrada al bloque ResNet
params.noise_dim = 26                                                                              # Dimension de la Capa de entrada
params.lr = 0.05                                                                                   # Tasa de aprendizaje del optimizador Adam (learning rate)
params.beta1 = 0.9                                                                                 # Coeficiente de decaimiento para el momento del primer orden del optimizador Adam
params.beta2 = 0.99                                                                                # Coeficiente de decaimiento para el momento del segundo orden del optimizador Adam
params.weight_decay = 0.001                                                                        # Termino de decaimiento del peso para regularizar los pesos del generador durante la optimizacion
params.step_size = 40000                                                                           # Numero de epicas despues de las cuales se reduce la tasa de aprendizaje
params.gamma = 0.5                                                                                 # El factor de reduccion para la tasa de aprendizaje. Despues de cada step_size epocas, la tasa de aprendizaje se multiplica por gamma


def  gausian(x, mean, sigma):
    return np.exp(- (x - mean) ** 2 / (2 * sigma ** 2))

def random_3gausians(x, min , max):
    x = 2 * math.pi / x * 1000
    a = np.random.uniform(min , max)
    b = np.random.uniform( 30 , 50)
    c = np.random.uniform(min , max)
    d = np.random.uniform( 10, 20)
    f = np.random.uniform(min , max)
    g = np.random.uniform( 30 , 70)
    
    return (gausian(x , a ,b) - gausian(x , c ,d)  + gausian(x , f ,g))** 2  / 4

def sixth_gausian(x, mean, sigma):
    return np.exp(- (x - mean) ** 6 / (2 * sigma ** 6))

thicknesses_list = []
ref_idx_list = []
Error = []
Delta_R = []
Sigma = []

figures_dir = 'figures'
if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)


for seed in range(1000):

    params.sigma = np.random.uniform(0.025, 0.05)
    Sigma.append(params.sigma)      
    params.condiciones = 2
    physicsparams = PhysicsParams(params.condiciones, user_define=True)

    physicsparams.n_bot_1 = 1
    physicsparams.n_top_1 = 1
    physicsparams.k_1 = 100
    physicsparams.k_values.append(physicsparams.k_1)
    physicsparams.lambda_min_1 = 100
    physicsparams.lambda_max_1 = 300
    physicsparams.theta_1 = 0
    physicsparams.pol_1 = "TE"

    physicsparams.n_bot_2 = 1
    physicsparams.n_top_2 = 1
    physicsparams.k_2 = 100
    physicsparams.k_values.append(physicsparams.k_2)
    physicsparams.theta_2 = 0
    physicsparams.pol_2 = "TE"


    a_b = 100
    mean = 300 + seed * 900 / 1000
      
    physicsparams.lambda_min_2 = mean - a_b
    physicsparams.lambda_max_2 = mean + a_b

    distancia_R = physicsparams.lambda_min_2 - physicsparams.lambda_min_1
    Delta_R.append(distancia_R)
    physicsparams.generate_physics_params()

    tarjet = Tarjet(params.condiciones, physicsparams.k_values, params.user_define)
    tarjet.configure_targets()

    tarjet_1_gaussian = random_3gausians(physicsparams.k_1, physicsparams.lambda_min_1, physicsparams.lambda_max_1)
    tarjet.tarjets["tarjet_1"].view(-1)[:] = tarjet_1_gaussian     

    tarjet_2_gaussian = random_3gausians(physicsparams.k_2, physicsparams.lambda_min_2, physicsparams.lambda_max_2)
    #tarjet_2_gaussian = sixth_gausian(physicsparams.k_2 , 2 * np.pi / mean * 1000 , sigma = 0.5)
    tarjet.tarjets["tarjet_2"].view(-1)[:] = tarjet_2_gaussian
    
    #___________________________________________________________________________________________________________
    
    torch.manual_seed(seed)
    glonet = GLOnet(params, physicsparams, tarjet)
    glonet.train()
    
    print(f"iteration{seed + 1 }")

    with torch.no_grad():
      params.k_test = 2 * math.pi / torch.linspace(0.3, 2.5, 50)
      params.theta_test = torch.linspace(0, math.pi/2.25, 50)
      (thicknesses, ref_index, result_mat) = glonet.evaluate(150, kvector=params.k_test, inc_angles=params.theta_test, grayscale=True)
      # Optimizacion
      
      reflex = MTMM_solver(params.condiciones, thicknesses, ref_index, physicsparams)
      FoM_reflex_total = sum(torch.pow(reflex[f'reflexion_{i}'] - tarjet.tarjets[f'tarjet_{i}'], 2).mean(dim=[1, 2, 3]) for i in range(1, 3))
      _, indices = torch.sort(FoM_reflex_total)
      opt_idx = indices[0]
      Error.append(FoM_reflex_total[opt_idx].detach().numpy())
      
      # Encontrar el índice óptimo

      optimal_thicknesses = thicknesses[opt_idx]
      optimal_ref_idx = ref_index[opt_idx]
      
      thicknesses_list.append(optimal_thicknesses.view(-1).cpu().numpy().tolist())
      ref_idx_list.append(optimal_ref_idx.view(-1).cpu().numpy().tolist())

    optimal_reflections = {}
          
 
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))  # 2 filas, 2 columnas
    
    # Subplot 1: tarjet_1 y tarjet_2
    
    axs[0, 0].plot(Delta_R, Error , "o-", color = "red",  label="error")
    axs[0, 0].set_title('Error vs Delta R', fontsize=18)
    axs[0, 0].set_xlabel('Delta_R', fontsize=14)
    axs[0, 0].set_ylabel('Error quadratico minimo', fontsize=14)
    axs[0, 0].legend()
    
    # Subplot 2: Pérdida durante el entrenamiento
    axs[0, 1].plot(glonet.loss_training)
    axs[0, 1].set_title('Training Loss', fontsize=18)
    axs[0, 1].set_ylabel('Loss', fontsize=14)
    axs[0, 1].set_xlabel('Iterations', fontsize=14)
    axs[0, 1].tick_params(axis='both', which='major', labelsize=12)
    
    # Subplot 3: Histograma de FoM
    axs[1, 0].hist(FoM_reflex_total.cpu().detach().numpy(), alpha=0.5)
    axs[1, 0].set_title(f'FoM Distribution (n\' = {1})', fontsize=18)
    axs[1, 0].set_xlabel('FoM', fontsize=14)
    axs[1, 0].tick_params(axis='both', which='major', labelsize=12)
    
    # Subplot 4: Reflexión óptima y tarjeta
    for i in range(1, 3):  # Reflexión 1 y 2
        reflex_key = f'reflexion_{i}'
        optimal_reflections[reflex_key] = reflex[reflex_key][opt_idx]

        tarjet_color = "violet" if i == 1 else "purple"
        axs[1, 1].plot(2 * math.pi / getattr(physicsparams, f'k_{i}') * 1000,
                       optimal_reflections[f'reflexion_{i}'][:, 0, 0].detach(),
                       "-", color=tarjet_color, label=f"Optimal Reflexion {i}")
    
        tarjet_color = "red" if i == 1 else "green"
        axs[1, 1].plot(2 * math.pi / getattr(physicsparams, f'k_{i}') * 1000,
                       tarjet.tarjets[f"tarjet_{i}"].view(-1),
                       ".-", color=tarjet_color, label=f"Tarjet Reflexion {i}")
    
    axs[1, 1].set_xlabel("Wavelength (nm)", fontsize=14)
    axs[1, 1].set_ylabel("Reflection", fontsize=14)
    axs[1, 1].legend(fontsize=10)
    axs[1, 1].set_title("Reflexions 1 and 2", fontsize=18)
    axs[1, 1].tick_params(axis='both', which='major', labelsize=12)
    
    # Ajuste de espacios entre subplots
    fig.savefig(os.path.join(figures_dir, f"results_seed_{seed}.png"))
    plt.close(fig)


with open('optimal_thicknesses.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(thicknesses_list)

# Archivo CSV para los índices de refracción
with open('optimal_ref_idx.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(ref_idx_list)

print(Error)
print(Delta_R)


  0%|          | 0/175 [00:00<?, ?it/s]

100%|██████████| 175/175 [00:19<00:00,  9.07it/s]


iteration1


100%|██████████| 175/175 [00:13<00:00, 13.31it/s]


iteration2


...

iteration86


100%|██████████| 175/175 [00:13<00:00, 13.04it/s]


iteration87


 71%|███████▏  | 125/175 [00:09<00:03, 12.98it/s]


KeyboardInterrupt: 